Please read the assignment description document.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports: No other imports allowed

In [2]:
import numpy as np #No using automatic differentiation allowed from here!
import pandas as pd 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import sklearn.metrics as metrics

In [3]:
def linear(Z):
    return Z

def tanh(x):
    return np.tanh(x)

def reLu(Z):
    return np.maximum(Z,0)

def softmax(Z):
    eA = np.exp(Z)
    return eA/np.sum(eA,axis=0)

In [4]:
def forward(W,A,B):
    Z = np.dot(W,A) + B
    
    assert(Z.shape == (W.shape[0],A.shape[1]))
    
    return Z

In [5]:
class LayerInfo():
    def __init__(self,ni,ni_1,function):
#         W,B,A,ni,ni_1
        print(ni,ni_1,function)
        self.F = function
        self.ni = ni
        self.ni_1 = ni_1
        self.A = None
        self.Z = None
        self.W = np.random.randn(ni,ni_1)*0.001
        self.B = np.zeros(shape=(ni,1))
        
    def getA(self,Ai_1):
        self.Z = forward(self.W,Ai_1,self.B)
        if self.F == 'linear':
            self.A = linear(self.Z)
            return linear(self.Z)
        elif self.F == 'tanh':
            self.A = tanh(self.Z)
            return tanh(self.Z)
        elif self.F == 'relu':
            self.A = reLu(self.Z)
            return reLu(self.Z)
        elif self.F == 'softmax':
            self.A = softmax(self.Z)
            return softmax(self.Z)
        
    def getFprime(self):
        if self.F == 'linear':
            return np.ones(shape = self.Z.shape,dtype = np.float32)
        elif self.F == 'relu':
            return np.array(self.Z>0,dtype = np.float32)
        elif self.F == 'tanh':
            return (1 - np.power(np.tanh(self.Z), 2))
            

## Class template

In [6]:
class NeuralNetworkClassifier: 
    def __init__(self, layers): 
        #### CHANGE THE CODE BELOW  
#         [(NUM_COLUMNS_X, "relu"), (200, "relu"), (NUM_CLASSES, "softmax")]
        self.Layers = []
        self.AList = []
        x = 0
        for i in layers:
            if x==0:
                l = LayerInfo(i[0],i[0],i[1])
                self.Layers.append(l)
                x+=1
            else :
                l = LayerInfo(layers[x][0],layers[x-1][0],layers[x][1])
                self.Layers.append(l)
                x+=1
        #### CHANGE THE CODE ABOVE
    def predict(self, X): 
        #### CHANGE THE CODE BELOW 
#         propagateForward
        self.AList = []
        A = X
        for l in self.Layers:
            A = l.getA(A)
        i = 0
        for l in self.Layers:
            self.AList.append(l.A)
            i+=1
        return A
        #### CHANGE THE CODE ABOVE
    def fit_once(self, X, y,alpha): 
        #### CHANGE THE CODE BELOW
        
        # propagate forward.
        self.predict(X)
        
        # propagate back.
        # update weights of layers using gradient descent.
        m = X.shape[1]
        i = len(self.Layers)-1
        dZ = (self.AList[i] - y)
        dW = (1/m)* np.dot(dZ,self.Layers[i-1].A.T)
        dB = (1/m)* dZ.sum(axis = 1,keepdims=True)
        
        self.Layers[i].W = self.Layers[i].W - (alpha*dW)
        self.Layers[i].B = self.Layers[i].B - (alpha*dB)
        i-=1
        
        while(i>=0):
            dZ = np.multiply(np.dot(self.Layers[i+1].W.T,dZ),self.Layers[i].getFprime())
            if i==0:
                dW = (1/m)*np.dot(dZ,X.T)
            else:
                dW = (1/m)*np.dot(dZ,self.Layers[i-1].A.T)
            dB = (1/m)*dZ.sum(axis=1,keepdims=True)
            
            self.Layers[i].W = self.Layers[i].W - (alpha*dW)
            self.Layers[i].B = self.Layers[i].B - (alpha*dB)
            i-=1
        
        #### CHANGE THE CODE ABOVE
    def categorical_cross_entropy_loss(self, y, yhat):
        #### CHANGE THE CODE BELOW
        m = y.shape[1]
        cost = (-np.sum(y*np.log(yhat+1e-9)))/m
        return cost
        #### CHANGE THE CODE ABOVE
        


## Code for Kaggle Competition

[link](https://www.kaggle.com/c/digit-recognizer/overview)

In [7]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv',)

In [8]:
y_train = train_df['label']
train_df.drop(columns = 'label',inplace = True)

In [9]:
train_df.isnull().sum()

In [10]:
x_train = train_df.to_numpy()

In [11]:
x_train = x_train.T

In [12]:
y_train = pd.get_dummies(data = y_train)

In [13]:
y_train = y_train.to_numpy()

In [14]:
y_train = y_train.T

In [15]:
y_train.shape

In [16]:
import matplotlib.pyplot as plt
import random
index = random.randrange(0, x_train.shape[1])
plt.imshow(x_train[:, index].reshape(28, 28), cmap = 'gray')
plt.show()

In [26]:
model = NeuralNetworkClassifier([(784, "tanh"),(10, "softmax")])

In [27]:
losses = [] 
iteration = 100
alpha = 0.01
for i in range(iteration):
    yhat = model.predict(x_train)
    loss = model.categorical_cross_entropy_loss(y_train, yhat)
    losses.append(loss)
    model.fit_once(x_train, y_train,alpha)
    if (i%(iteration/10))==0:
        print(" cost at "+str(i)+" iteration is "+str(loss))

In [28]:
t = np.arange(0, NUM_ITERS)
plt.plot(t, losses)
plt.show()

In [29]:
def accuracy(inp,y_actual,model):
    y = model.predict(inp)
    y = np.argmax(y,0)
    y_actual = np.argmax(y_actual,0)
    acc  = np.mean(y==y_actual)*100
    return acc

In [30]:
print("Accuracy of Train Dataset", accuracy(x_train, y_train, model), "%")

In [62]:
Y_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [63]:
# Y_test = pd.get_dummies
Y_test = pd.get_dummies(data = Y_test)
Y_test= Y_test.T
Y_pred = model.predict(Y_test)
Y_pred = np.argmax(Y_pred,0)
Y_pred = Y_pred.T

In [64]:
y_pred = pd.DataFrame(Y_pred,columns = ['label'])
y_pred.to_csv("finalPred.csv")

In [67]:
df = pd.read_csv("./finalPred.csv")

In [69]:
df.rename(columns={'Unnamed: 0': 'ImageId', 'label': 'Label'}, inplace=True)

In [76]:
df.head()
for i in range(28000):
    df.loc[i,'ImageId']=i+1
# df.loc[row_index,col_indexer] = value    

In [78]:
df.to_csv("finalPred.csv",index=False)